# kaggle - Intro to SQL & Advanced SQL

https://www.kaggle.com/learn/intro-to-sql  
https://www.kaggle.com/learn/advanced-sql

- [1. Getting Stared With SQL and BigQuery](#1.-Getting-Stared-With-SQL-and-BigQuery)
    - [1.1 Introduction](#1.1-Introduction)
    - [1.2 First BigQuery commands](#1.2-First-BigQuery-commands)
    - [1.3 Table schema](#1.3-Table-schema)
- [2. Select, From & Where](#2.-Select,-From-&-Where)
    - [2.1 Introduction](#2.1-Introduction)
    - [2.2 SELECT ... FROM](#2.2-SELECT-...-FROM)
    - [2.3 WHERE ...](#2.3-WHERE-...)
- [3. Group by, Having & Count](#3.-Group-by,-Having-&-Count)
    - [3.1 COUNT()](#3.1-COUNT())
    - [3.2 GROUP BY](#3.2-GROUP-BY)
    - [3.3 GROUP BY ... HAVING](#3.3-GROUP-BY-...-HAVING)
    - [3.4 Aliasing and other improvements](#3.4-Aliasing-and-other-improvements)
    - [3.5 Note on using GROUP BY](#3.5-Note-on-using-GROUP-BY)



## 1. Getting Stared With SQL and BigQuery


### 1.1 Introduction
__Structured Query Language__ (SQL), is the programming language used with databases, and it is an important skill for any data scientist. In this course, you'll build your SQL skills using __BigQuery__, a web service that lets you apply SQL to huge datasets.

### 1.2 First BigQuery commands

Import BigQuery Python package
> ```python
from google.cloud import bigquery
>```

The first step in the workflow is to create a `Client` object. 
> ```python 
> # Create a "Client" object
client = bigquery.Client()
> ```

We'll work with a dataset of posts on [Hacker News](https://news.ycombinator.com/), a website focusing on computer science and cybersecurity news.

In BigQuery, each dataset is contained in a corresponding project. In this case, our `hacker_news` dataset is contained in the `bigquery-public-data` project. To access the dataset,
- We begin by constructing a reference to the dataset with the `dataset()` method.
- Next, we use the `get_dataset()` method, along with the reference we just constructed, to fetch the dataset.

> ```python
> # Construct a reference to the "hacker_news" dataset
> dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")  
>
> # API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
> ```

Every dataset is just a collection of tables. You can think of a dataset as a spreadsheet file containing multiple tables, all composed of rows and columns.  
We use the `list_tables()` method to list the tables in the dataset.

> ```python
> # List all the tables in the "hacker_news" dataset
> tables = list(client.list_tables(dataset))
> 
> # Print names of all tables in the dataset (there are four!)
> for table in tables:  
>     print(table.table_id)
> ```

<p style="background:black">
<code style="background:black;color:white">comments
full
full_201510
stories
</code>
</p>

Similar to how we fetched a dataset, we can fetch a table. In the code cell below, we fetch the `full` table in the `hacker_news` dataset.

> ```python
> # Construct a reference to the "full" table
> table_ref = dataset_ref.table("full")
> 
> # API request - fetch the table
> table = client.get_table(table_ref)
> ```

<!-- ![BigQuery](img/bigquery.png "BigQuery") -->
<img src="img/bigquery.png" alt="Drawing" title="BigQuery" style="width: 800px;"/>

### 1.3 Table schema
The structure of a table is called its **schema**. We need to understand a table's schema to effectively pull out the data we want.

> ```python
> # Print information on all the columns in the "full" table in the "hacker_news" dataset
> table.schema
> ```

<p style="background:black">
<code style="background:black;color:white">[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', (), None),
 ...,
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', (), None)]
</code>
</p>

Each `SchemaField` tells us about a specific column (which we also refer to as a field). In order, the information is:
- **name** of the column
- **field type** (or **datatype**) in the column
- **mode** of the column ('NULLABLE' means that a column allows NULL values, and is the default)
- **description** of the data in that column

The first field has the SchemaField:
<p style="background:black">
<code style="background:black;color:white">SchemaField('by', 'string', 'NULLABLE', "The username of the item's author.",())
</code>
</p>

- the field (or column) is called by,
- the data in this field is strings,
- NULL values are allowed, and
- it contains the usernames corresponding to each item's author.

We can use the `list_rows()` method to check just the first five lines of of the full table to make sure this is right. (Sometimes databases have outdated descriptions, so it's good to check.) This returns a BigQuery `RowIterator` object that can quickly be converted to a pandas DataFrame with the `to_dataframe()` method.

> ```python
> # Preview the first five lines of the "full" table
> client.list_rows(table, max_results=5).to_dataframe()
> ```

The list_rows() method will also let us look at just the information in a specific column. If we want to see the first five entries in the by column, for example, we can do that!

> ```python
> # Preview the first five entries in the "by" column of the "full" table
> client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()
> ```

## 2. Select, From & Where

### 2.1 Introduction

We'll work with a small imaginary dataset `pet_records` which contains just one table, called pets.

| ID | Name | Animal| 
| --- | --- | --- | 
| 1 | Dr. Harris Bonkers | Rabbit | 
| 2 | Moon | Dog|
| 3 | Ripley | Cat |
| 4 | Tom | Cat |

### 2.2 SELECT ... FROM

Select a single column
- `SELECT` specifies the column
- `FROM` specifies the table

> ```python
SELECT Name
FROM `bigquery-public-data.pet_records.pets`
> ```

Select multiple columns
> ```python
SELECT Name, Animal
FROM `bigquery-public-data.pet_records.pets`
> ```

Select all columns
> ```python
SELECT *
FROM `bigquery-public-data.pet_records.pets`
> ```

<div class="alert alert-block alert-info"><b>Note:</b> when writing an SQL query, the argument we pass to FROM is not in single or double quotation marks (' or "). It is in backticks (`).</div>

### 2.3 WHERE ...

Return only the rows meeting specific conditions using the `WHERE` clause.

> ```python
SELECT Name
FROM `bigquery-public-data.pet_records.pets`
WHERE Animal='Cat'
> ```

## 3. Group by, Having & Count

### 3.1 COUNT()

`COUNT()` returns a count of things. If you pass it the name of a column, it will return the number of entries in that column.

> ```python
SELECT COUNT(ID)
FROM `bigquery-public-data.pet_records.pets`
> ```

| f0_ |
| --- | 
| 4 | 


`COUNT()` is an example of an **aggregate function**, which takes many values and returns one.  
Other examples: `SUM()`, `AVG()`, `MIN()`, `MAX()`.

### 3.2 GROUP BY

`GROUP BY` takes the name of one or more columns, and treats all rows with the same value in that column as a single group when you apply aggregate functions like `COUNT()`.

How many of each type of animal in the pets table? We can use `GROUP BY` to group together rows that have the same value in the `Animal` column, while using `COUNT()` to find out how many ID's we have in each group.

> ```python
SELECT Animal, COUNT(ID)
FROM `bigquery-public-data.pet_records.pets`
GROUP BY Animal
> ```

| Animal | f0_ |
| ------ | --- |  
| Rabbit | 1 |
| Dog | 1 |
| Cat | 2 |

### 3.3 GROUP BY ... HAVING

`HAVING` is used in combination with `GROUP BY` to ignore groups that don't meet certain criteria.

Only include groups that have more than one ID in them

> ```python
SELECT Animal, COUNT(ID)
FROM `bigquery-public-data.pet_records.pets`
GROUP BY Animal
HAVING COUNT(ID)>1
> ```

| Animal | f0_ |
| ------ | --- |  
| Cat | 2 |

### 3.4 Aliasing and other improvements

- Aliasing: adding `AS NewName` after you specify the aggregation replaces the column name `f0__`
- If you are ever unsure what to put inside the `COUNT()` function, you can do `COUNT(1)` to count the rows in each group. Most people find it especially readable, because we know it's not focusing on other columns. It also scans less data than if supplied column names (making it faster and using less of your data access quota).

> ```python
SELECT Animal, COUNT(1) AS NumPets
FROM `bigquery-public-data.pet_records.pets`
GROUP BY Animal
HAVING COUNT(1)>1
> ```

| Animal | NumPets |
| ------ | --- |  
| Cat | 2 |

### 3.5 Note on using GROUP BY

Note that because it tells SQL how to apply aggregate functions (like `COUNT()`), it doesn't make sense to use `GROUP BY` without an aggregate function. Similarly, if you have any `GROUP BY` clause, then all variables must be passed to either a
1. `GROUP BY` command, or
2. an aggregation function.


> ```python
SELECT ID, Animal, COUNT(1) AS NumPets
FROM `bigquery-public-data.pet_records.pets`
GROUP BY Animal
HAVING COUNT(1)>1
> ```

<p style="background:black">
<code style="background:black;color:red">SELECT list expression references column (column's name) which is neither grouped nor aggregated at
</code>
</p>


## 4. Order by

### 4.1 ORDER BY

`ORDER BY` is usually the last clause in your query, and it sorts the results returned by the rest of your query.

> ```python
SELECT ID, Name, Animal
FROM `bigquery-public-data.pet_records.pets`
ORDER BY ID
> ```

| ID | Name | Animal| 
| --- | --- | --- | 
| 1 | Dr. Harris Bonkers | Rabbit | 
| 2 | Moon | Dog|
| 3 | Ripley | Cat |
| 4 | Tom | Cat |

The `ORDER BY` clause also works for columns containing text, where the results show up in alphabetical order.

> ```python
SELECT ID, Name, Animal
FROM `bigquery-public-data.pet_records.pets`
ORDER BY Animal
> ```

| ID | Name | Animal| 
| --- | --- | --- | 
| 4 | Tom | Cat |
| 3 | Ripley | Cat |
| 2 | Moon | Dog|
| 1 | Dr. Harris Bonkers | Rabbit | 

You can reverse the order using the `DESC` argument (default `ASC`).

> ```python
SELECT ID, Name, Animal
FROM `bigquery-public-data.pet_records.pets`
ORDER BY Animal DESC
> ```

| ID | Name | Animal| 
| --- | --- | --- | 
| 1 | Dr. Harris Bonkers | Rabbit | 
| 2 | Moon | Dog|
| 3 | Ripley | Cat |
| 4 | Tom | Cat |

### 4.2 Dates

There are two ways that dates can be stored in BigQuery: as a **DATE** or as a **DATETIME**.

The **DATE** format has the year first, then the month, and then the day. It looks like this:  

`YYYY-[M]M-[D]D`
- YYYY: Four-digit year
- [M]M: One or two digit month
- [D]D: One or two digit day

The **DATETIME** format is like the date format ... but with time added at the end.

### 4.3 EXTRACT

Often you'll want to look at part of a date, like the year or the day. You can do this with `EXTRACT`. Table `pets_with_date`.

| ID | Name | Animal| Date |
| --- | --- | --- | --- |
| 1 | Dr. Harris Bonkers | Rabbit | 2019-04-18 | 
| 2 | Moon | Dog| 2019-05-16 |
| 3 | Ripley | Cat | 2019-01-07 |
| 4 | Tom | Cat | 2019-02-23 |

> ```python
SELECT Name, EXTRACT(DAY from Date) AS Day
FROM `bigquery-public-data.pet_records.pets_with_date`
> ```

| Name | Day |
| --- | --- |
| Dr. Harris Bonkers | 18 | 
| Moon | 16 |
| Ripley | 7 |
| Tom | 23 |

[Date and time functions](https://cloud.google.com/bigquery/docs/reference/legacy-sql#datetimefunctions)

> ```python
SELECT Name, EXTRACT(WEEK from Date) AS Week
FROM `bigquery-public-data.pet_records.pets_with_date`
> ```

| Name | Week |
| --- | --- |
| Dr. Harris Bonkers | 15 | 
| Moon | 19 |
| Ripley | 1 |
| Tom | 7 |

> ```python
> # Query to find out the number of accidents for each day of the week
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """
> ```